Project is inspired by [Analytics Vidhya](https://www.analyticsvidhya.com/blog/2021/09/data-analysis-and-price-prediction-of-electric-vehicles/) and is an update on this [Kaggle dataset](https://www.kaggle.com/datasets/kkhandekar/cheapest-electric-cars)

In [1]:
import numpy as np
import regex as re
import pandas as pd

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [831]:
with open('data/paths.txt') as path:
    paths = path.read().split('\n')

In [9]:
driver_path = paths[1]
brave_path = paths[2]
# extensions = open('../data/paths.txt','r').read().split('\n')
# u_block = extensions[2]

option = webdriver.ChromeOptions()
option.binary_location = brave_path
option.add_argument('--disable-dev-shm-usage')
option.add_experimental_option("detach", True)
# option.add_argument('--load-extension='+u_block)

# Create new Instance of Chrome
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
# enable browser logging
d = DesiredCapabilities.CHROME
d['goog:loggingPrefs'] = { 'browser':'ALL' }

In [11]:
# def seleniumCall():
browser = webdriver.Chrome(executable_path=driver_path, options=option, desired_capabilities=d);
wait = WebDriverWait(browser, timeout=10, poll_frequency=1)

url = 'https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~350|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all'
browser.get(url)
winds = browser.window_handles

browser.switch_to.window(winds[0])
browser.maximize_window()

C:\Users\Mohammed\AppData\Local\Temp\ipykernel_10384\1098924167.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path=driver_path, options=option, desired_capabilities=d);


In [4]:
def clickXP(string):
    element = wait.until(EC.presence_of_element_located((By.XPATH, string)))
    return element    

def selectXP(string):
    # element = browser.find_element(By.XPATH,string)
    element = wait.until(EC.presence_of_element_located((By.XPATH, string)))
    return element

In [12]:
clickXP('//*[@id="evdb"]/div[2]/div[2]/div[1]/div[2]/div[2]/button[1]/p').click()

In [39]:
full_list = selectXP('//*[@id="evdb"]/main/div[2]/div[3]')

In [40]:
lst = full_list.find_elements(By.CLASS_NAME,'data-wrapper')

In [74]:
dic_list = []

for item in lst:
    car = item.text.split('\n')

    dic = {}
    dic['title'] = car[0]
    dic['useable_battery'] = car[1]
    dic['date'] = car[2]
    
    tow_split = car[3].split(' ')
    if len(tow_split)>1:
        dic['towing'] = tow_split[0]
        dic['market_segment'] = tow_split[1][0]
        dic['seats'] = tow_split[1][1]
    else:
        dic['towing'] = np.nan
        dic['market_segment'] = tow_split[0][0]
        dic['seats'] =  tow_split[0][1]
            
    dic['acceleration'] = car[5]
    dic['max_speed'] = car[7]
    dic['real_range'] = car[9]
    dic['efficiency'] = car[11]
    dic['fastcharge'] = car[13]
    dic['price_germany'] = car[14]
    dic['price_ned'] = car[15]
    dic['price_uk'] = car[16]
    
    dic_list.append(dic)

In [341]:
ev_data = pd.DataFrame(data=dic_list,columns=list(dic.keys()))
ev_data.head()

,title,useable_battery,date,towing,market_segment,seats,acceleration,max_speed,real_range,efficiency,fastcharge,price_germany,price_ned,price_uk
0,Tesla Model Y Long Range Dual Motor,75 kWh useable battery*,Available since February 2022,1600,D,5,5.0 sec,217 km/h,435 km,172 Wh/km,670 km/h,"€59,017","€54,993","£52,990"
1,Tesla Model Y,57.5 kWh useable battery*,Available since November 2022,1600,D,5,6.9 sec,217 km/h,345 km,167 Wh/km,580 km/h,"€47,567","€47,993","£44,990"
2,Tesla Model 3,57.5 kWh useable battery*,Available since December 2021,1000,D,5,6.1 sec,225 km/h,380 km,151 Wh/km,630 km/h,"€46,667","€45,993","£42,990"
3,BYD ATTO 3,60 kWh useable battery*,Available since August 2022,NaN,C,5,7.3 sec,160 km/h,320 km,188 Wh/km,300 km/h,"€44,625","€42,998",N/A
4,Tesla Model 3 Long Range Dual Motor,75 kWh useable battery*,Available since November 2021,1000,D,5,4.4 sec,233 km/h,485 km,155 Wh/km,750 km/h,"€56,667","€53,993","£50,990"


In [98]:
print(sum(ev_data.useable_battery.str.contains('useable battery*')),
sum(ev_data.acceleration.str.contains('sec')),
sum(ev_data.max_speed.str.contains('km.h')),
sum(ev_data.real_range.str.contains('km')),
sum(ev_data.efficiency.str.contains('Wh/km')),
sum(ev_data.fastcharge.str.contains('km/h')))

298 298 298 298 298 295


In [173]:
sum(ev_data['date'].str.contains('since'))

275

In [176]:
sum(ev_data['date'].str.contains('from'))

23

In [296]:
def numPart(string):
    if string == '-':
        return np.nan
    else:
        return re.search('[0-9,.]*',string).group()

In [192]:
def getMonth(string):
    #first space index
    id1 = string.index(' ')
    #second space index
    id2 = id1 + string[id1+1:].index(' ')
    
    return string[id2+2:-5]

In [357]:
def getPrice(string):
    if string == 'N/A':
        return np.nan
    else:
        string = string.replace('€','')
        string = string.replace('£','')
        string = string.replace(',','')
        string = string.replace('*','')
        string = string.strip()
        
        return string

In [350]:
string_cols = ['manufacturer','model','released','release_month','release_year']
num_cols = ['useable_battery','acceleration','max_speed','real_range',
             'efficiency','fastcharge']
price_cols = ['price_germany','price_ned','price_uk']

In [358]:
ev_data = pd.DataFrame(data=dic_list,columns=list(dic.keys()))
ev_data['manufacturer'] = ev_data['title'].apply(lambda x: x[:x.index(' ')])
ev_data['model'] = ev_data['title'].apply(lambda x: x[x.index(' ' ):])
ev_data['released'] = ev_data['date'].apply(lambda x: 1 if 'since' in x else 0)
ev_data['release_month'] = ev_data['date'].apply(lambda x: getMonth(x))
ev_data['release_year'] = ev_data['date'].apply(lambda x: x[-4:])

for col in num_cols:
    ev_data[col] = np.vectorize(numPart)(ev_data[col])
    
for price in price_cols:
    ev_data[price] = ev_data[price].apply(lambda x: getPrice(x))
    
for col in ['fastcharge']+price_cols:
    ev_data[col] = ev_data[col].apply(lambda x: np.nan if x=='nan' else x) 

col_order = string_cols+num_cols+price_cols

ev_data = ev_data[col_order]

In [359]:
for col in num_cols:
    try:
        ev_data[col] = ev_data[col].astype('Int64')
    except:
        ev_data[col] = ev_data[col].astype('float')

In [363]:
for price in price_cols:
    ev_data[price] = ev_data[price].astype('Int64')

In [377]:
ev_data[string_cols] = ev_data[string_cols].astype('category')

In [812]:
ev_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298 entries, 0 to 297
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   manufacturer     298 non-null    category
 1   model            298 non-null    object  
 2   released         298 non-null    category
 3   release_month    298 non-null    category
 4   release_year     298 non-null    category
 5   useable_battery  298 non-null    float64 
 6   acceleration     298 non-null    float64 
 7   max_speed        298 non-null    Int64   
 8   real_range       298 non-null    Int64   
 9   efficiency       298 non-null    Int64   
 10  fastcharge       295 non-null    Int64   
 11  price_germany    273 non-null    Int64   
 12  price_ned        266 non-null    Int64   
 13  price_uk         187 non-null    Int64   
dtypes: Int64(7), category(4), float64(2), object(1)
memory usage: 28.8+ KB


In [813]:
ev_data.to_csv('../data/ev_data.csv')